# Ordering and Delivery

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/planetlabs/training-workshop/blob/friedemann_colab_updates/OrdersAPI/ordering_and_delivery.ipynb)

This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list, then we create an order for two Analytic `PSScene4Band` images. We poll for order success then download images individually. And finally, we create, poll, and download the same order delivered as a single zip file.

Reference information can be found at [Ordering & Delivery](https://developers.planet.com/docs/orders/ordering-delivery/).


In [ ]:
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth


## Authenticating

<u>Option 1: Using Google Colab</u>


In [ ]:
# Import the os module in order to access environment variables
from google.colab import drive

# In order to prevent making your API key visible to others,  it makes sense to
# store your API key as a *.json file in your Google drive (in this case in a subfolder named "Colab Notebooks")
# Make sure to allow Colab to connect to your Google Drive.
drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/Colab Notebooks")
with open("PLANET_API_KEY.json") as json_file:
    PLANET_API_KEY = json.load(json_file)["PL_API_KEY"]


<u>Option 2: Run local Jupyter instance</u>


In [ ]:
# API Key stored as an env variable
PLANET_API_KEY = os.getenv("PL_API_KEY")


### Set up the main endpoint of the Orders API


In [ ]:
orders_url = "https://api.planet.com/compute/ops/orders/v2"


### Curl example

To check your orders list and make sure you have the permissions you need, uncomment the following line to run `curl`


In [ ]:
# !curl -L -H "Authorization: api-key $PLANET_API_KEY" $orders_url


### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.


In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, "")
response = requests.get(orders_url, auth=auth)
response


Now we will list the orders we have created thus far. Your list may be empty if you have not created an order yet.


In [ ]:
orders = response.json()["orders"]
len(orders)


## Ordering

In this example, we will order two `PSScene4Band` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/docs/orders/ordering-delivery/#ordering-data_1).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the `PSScene4Band` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

### Place Order


In [ ]:
# set content type to json
headers = {"content-type": "application/json"}


In [ ]:
request = {
    "name": "simple order",
    "products": [
        {
            "item_ids": [
                "20220921_051448_33_2475",
                "20220921_051258_63_2475",
                "20220921_051254_06_2475",
                "20220921_051249_49_2475",
                "20220921_051244_92_2475",
                "20220921_051240_35_2475",
                "20220921_051235_78_2475",
                "20220921_051231_21_2475",
                "20220921_051226_64_2475",
                "20220921_051222_07_2475",
                "20220921_051217_50_2475",
                "20220921_051212_92_2475",
                "20220921_051208_35_2475",
                "20220921_051154_64_2475",
                "20220921_051002_66_2475",
                "20220921_050958_09_2475",
                "20220921_050955_80_2475",
                "20220921_050951_23_2475",
                "20220921_050946_66_2475",
                "20220921_050944_37_2475",
            ],
            "item_type": "PSScene",
            "product_bundle": "visual",
        }
    ],
}


In [ ]:
def place_order(request, auth):
    response = requests.post(
        orders_url, data=json.dumps(request), auth=auth, headers=headers
    )
    if response.status_code > 299:
        print(response.text)
    order_id = response.json()["id"]
    print(order_id)
    order_url = orders_url + "/" + order_id
    return order_url


In [ ]:
order_url = place_order(request, auth)


### Cancel an Order


In [ ]:
# report order state
requests.get(order_url, auth=auth).json()["state"]


In [ ]:
# oid = oids[0]
response = requests.put(order_url, auth=auth)
response


In [ ]:
# report order state - it could take a little while to cancel
requests.get(order_url, auth=auth).json()["state"]


### Poll for Order Success


In [ ]:
# re-order since we canceled our last order
order_url = place_order(request, auth)


In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while count < num_loops:
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response["state"]
        print(state)
        end_states = ["success", "failed", "partial"]
        if state in end_states:
            break
        time.sleep(5)


poll_for_success(order_url, auth)


### View Results


In [ ]:
# If you don't want to wait for your order to succeed before running this cell, you can recreate the order_url based on
# an order you created earlier. To do so, uncomment the following line and paste the replace the order id.

# order_url = orders_url + "/e56999c3-c03d-479a-be0d-9b140cd6f323"

r = requests.get(order_url, auth=auth)
response = r.json()
results = response["_links"]["results"]


In [ ]:
[r["name"] for r in results]


## Download

### Downloading each asset individually


In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r["location"] for r in results]
    results_names = [r["name"] for r in results]
    print("{} items to download".format(len(results_urls)))

    # Create an output sub directory
    current_dir = os.getcwd()
    out_dir = current_dir + "/output"

    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join(out_dir, name))

        if overwrite or not path.exists():
            print("downloading {} to {}".format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, "wb").write(r.content)
        else:
            print("{} already exists, skipping {}".format(path, name))


In [ ]:
download_results(results)


### Downloading as a single zip

To download all of the order assets as a single zip, the order request needs to be changed slightly with delivery instructions. After that, polling and downloading are the same.


In [ ]:
zip_delivery = {"delivery": {"single_archive": True, "archive_type": "zip"}}
request_zip = request.copy()
request_zip.update(zip_delivery)
request_zip


In [ ]:
order_url = place_order(request_zip, auth)


In [ ]:
poll_for_success(order_url, auth)


In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response["_links"]["results"]


In [ ]:
download_results(results)
